# Used car prices - Autotrade

## Data Cleaning

#### Data source: https://www.autotrader.com.au/
#### Scope: Perth WA, 25km radius from postcode 6000

In [667]:
# Import libraries
import numpy as np
import pandas as pd
from pathlib import Path
import numpy as np
import requests
import time
import json
import math

# Import the API key
from config import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

##### Load the data

In [668]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(Path('data_collection_output/output_from_1_to_251.csv'))

# Review the DataFrame
df.head()

,Year model,Car Spec,Kilometres,Seller type,Price,Transmission,Body type,Drive type,Engine,Fuel type,...,Make,Model,Variant,Series,Warranty when new (months),Warranty when new (kms),Service interval (months),Service interval (kms),Country of origin,Vehicle segment
0,2007,2007 Audi TT 8J Coupe 2dr S tronic 6sp 2.0T,"104,108km",Dealer: Used,"$17,990",Sports Automatic Dual Clutch,-,-,-,NaN,...,Audi,TT,8J Coupe 2dr S tronic 6sp 2.0T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,2019 Hyundai I30 Active PD2 MY19,"74,070km",Dealer: Used,"$20,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,"4 cyl, 2 L",Unleaded Petrol,...,Hyundai,I30,Active,PD2 MY19,60.0,"999,000",12.0,"15,000",NaN,NaN
2,2019,2019 Hyundai I30 Active PD2 MY19,"66,975km",Dealer: Used,"$21,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,"4 cyl, 2 L",Unleaded Petrol,...,Hyundai,I30,Active,PD2 MY19,60.0,"999,000",12.0,"15,000",NaN,NaN
3,2019,2019 Hyundai I30 Active PD2 MY19,"68,845km",Dealer: Used,"$21,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,"4 cyl, 2 L",Unleaded Petrol,...,Hyundai,I30,Active,PD2 MY19,60.0,"999,000",12.0,"15,000",NaN,NaN
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,"188,978km",Dealer: Used,"$24,750",Automatic,"Ute Tray, 4 Doors, 5 Seats",4x4,"4 cyl, 2.4 L",Diesel,...,Mitsubishi,Triton,GLX (4X4),MQ MY18,60.0,"100,000",12.0,"15,000",NaN,NaN


##### Clean the data

In [669]:
# Check initial data info types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5670 entries, 0 to 5669
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year model                   5670 non-null   int64  
 1   Car Spec                     5659 non-null   object 
 2   Kilometres                   5659 non-null   object 
 3   Seller type                  5659 non-null   object 
 4   Price                        5659 non-null   object 
 5   Transmission                 5659 non-null   object 
 6   Body type                    5659 non-null   object 
 7   Drive type                   5659 non-null   object 
 8   Engine                       5659 non-null   object 
 9   Fuel type                    5305 non-null   object 
 10  Fuel consumption             5659 non-null   object 
 11  Colour ext / int             5659 non-null   object 
 12  Registration                 5659 non-null   object 
 13  Rego expiry       

In [670]:
# Drop columns no needed
df_clean = df.drop(['Seller type', 'Engine', 'Registration', 'Rego expiry', 'Stock No', 'ANCAP Safety rating', 'Green overall rating', 'Front tyre size',
       'Front rim size', 'Rear tyre size', 'Rear rim size', 'Injection / Carburation','Front suspension', 'Rear suspension', 'Front brakes', 'Rear brakes',
       'CO2 level (g/km)', 'Green house rating', 'Overall HxWxL', 'Ground clearance unladen', 'Wheelbase', 'Turning circle', 'Rear track', 'Front track',
       'Gross trailer weight braked', 'Series', 'Warranty when new (months)', 'Warranty when new (kms)', 'Service interval (months)', 'Service interval (kms)',
       'Country of origin', 'Vehicle segment'], axis=1)

df_clean.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption,Colour ext / int,...,CC,Number of cylinders,Fuel tank capacity,Valve gear type,Maximum torque,Maximum power (kW),Kerb weight,Make,Model,Variant
0,2007,2007 Audi TT 8J Coupe 2dr S tronic 6sp 2.0T,"104,108km","$17,990",Sports Automatic Dual Clutch,-,-,NaN,-,Blue / -,...,NaN,NaN,NaN,NaN,-,-,-,Audi,TT,8J Coupe 2dr S tronic 6sp 2.0T
1,2019,2019 Hyundai I30 Active PD2 MY19,"74,070km","$20,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
2,2019,2019 Hyundai I30 Active PD2 MY19,"66,975km","$21,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
3,2019,2019 Hyundai I30 Active PD2 MY19,"68,845km","$21,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,"188,978km","$24,750",Automatic,"Ute Tray, 4 Doors, 5 Seats",4x4,Diesel,7.60 L / 100 km,White / Black,...,2440.0,4.0,75.0,16.0,430 @ 2500 RPM,133 @ 3500 RPM,1930 kg,Mitsubishi,Triton,GLX (4X4)


In [671]:
# Drop null rows and duplicates
df_clean = df_clean.drop_duplicates(subset="VIN")
df_clean = df_clean.dropna()
df_clean.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption,Colour ext / int,...,CC,Number of cylinders,Fuel tank capacity,Valve gear type,Maximum torque,Maximum power (kW),Kerb weight,Make,Model,Variant
1,2019,2019 Hyundai I30 Active PD2 MY19,"74,070km","$20,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
2,2019,2019 Hyundai I30 Active PD2 MY19,"66,975km","$21,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
3,2019,2019 Hyundai I30 Active PD2 MY19,"68,845km","$21,998",Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,"188,978km","$24,750",Automatic,"Ute Tray, 4 Doors, 5 Seats",4x4,Diesel,7.60 L / 100 km,White / Black,...,2440.0,4.0,75.0,16.0,430 @ 2500 RPM,133 @ 3500 RPM,1930 kg,Mitsubishi,Triton,GLX (4X4)
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,"141,855km","$26,900",Automatic,"Ute Tray, 4 Doors, 5 Seats",4x4,Diesel,7.60 L / 100 km,White / Black,...,2440.0,4.0,75.0,16.0,430 @ 2500 RPM,133 @ 3500 RPM,1930 kg,Mitsubishi,Triton,GLX (4X4)


In [672]:
# Check data info
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2163 entries, 1 to 5669
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year model           2163 non-null   int64  
 1   Car Spec             2163 non-null   object 
 2   Kilometres           2163 non-null   object 
 3   Price                2163 non-null   object 
 4   Transmission         2163 non-null   object 
 5   Body type            2163 non-null   object 
 6   Drive type           2163 non-null   object 
 7   Fuel type            2163 non-null   object 
 8   Fuel consumption     2163 non-null   object 
 9   Colour ext / int     2163 non-null   object 
 10  VIN                  2163 non-null   object 
 11  Dealer               2163 non-null   object 
 12  Address              2163 non-null   object 
 13  Seating capacity     2163 non-null   float64
 14  Doors                2163 non-null   float64
 15  CC                   2163 non-null   f

In [673]:
# Convert Kilometres and price to integer
df_clean['Kilometres'] = pd.to_numeric(df_clean['Kilometres'].str.replace(',', '').str.replace('km', ''), errors='coerce')
df_clean['Price'] = pd.to_numeric(df_clean['Price'].str.replace('$', '').str.replace(',', ''), errors='coerce')
df_clean.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption,Colour ext / int,...,CC,Number of cylinders,Fuel tank capacity,Valve gear type,Maximum torque,Maximum power (kW),Kerb weight,Make,Model,Variant
1,2019,2019 Hyundai I30 Active PD2 MY19,74070.0,20998,Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
2,2019,2019 Hyundai I30 Active PD2 MY19,66975.0,21998,Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
3,2019,2019 Hyundai I30 Active PD2 MY19,68845.0,21998,Automatic,"HATCH, 4 Doors, 5 Seats",Front Wheel Drive,Unleaded Petrol,7.40 L / 100 km,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,188978.0,24750,Automatic,"Ute Tray, 4 Doors, 5 Seats",4x4,Diesel,7.60 L / 100 km,White / Black,...,2440.0,4.0,75.0,16.0,430 @ 2500 RPM,133 @ 3500 RPM,1930 kg,Mitsubishi,Triton,GLX (4X4)
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,141855.0,26900,Automatic,"Ute Tray, 4 Doors, 5 Seats",4x4,Diesel,7.60 L / 100 km,White / Black,...,2440.0,4.0,75.0,16.0,430 @ 2500 RPM,133 @ 3500 RPM,1930 kg,Mitsubishi,Triton,GLX (4X4)


In [674]:
# Validate for null values found in the previous result
print(df_clean['Kilometres'].isnull().value_counts())
print(df_clean['Price'].isnull().value_counts())

False    2158
True        5
Name: Kilometres, dtype: int64
False    2163
Name: Price, dtype: int64


In [675]:
# Drop the null values
df_clean.dropna(inplace=True)

In [676]:
# Check data info
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2158 entries, 1 to 5669
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year model           2158 non-null   int64  
 1   Car Spec             2158 non-null   object 
 2   Kilometres           2158 non-null   float64
 3   Price                2158 non-null   int64  
 4   Transmission         2158 non-null   object 
 5   Body type            2158 non-null   object 
 6   Drive type           2158 non-null   object 
 7   Fuel type            2158 non-null   object 
 8   Fuel consumption     2158 non-null   object 
 9   Colour ext / int     2158 non-null   object 
 10  VIN                  2158 non-null   object 
 11  Dealer               2158 non-null   object 
 12  Address              2158 non-null   object 
 13  Seating capacity     2158 non-null   float64
 14  Doors                2158 non-null   float64
 15  CC                   2158 non-null   f

In [677]:
# Check transmission value counts to do binning of small values
df_clean['Transmission'].value_counts()

Automatic                  1093
Continuous Variable         344
Manual                      305
Auto Dual Clutch             93
Auto Direct Shift            62
Auto Seq Sportshift          33
CVT Auto Sequential          31
Direct Shift                 26
Auto Sequential              20
Auto D/SH T/Tron Spt         15
Auto Sports Mode             15
Auto SEQ Sports Mode         13
Automated Manual             11
Automatic G-Tronic            9
Automatic Tiptronic           9
Sequential Auto               9
Auto Activematic              9
-                             6
Manual 4x4                    6
Automatic Geartronic          6
Auto S-Tronic                 6
Auto Elec Sportshift          5
Auto Steptronic Sport         5
Auto Active Sequential        5
CVT Multitronic               3
Auto Sports Shift             3
Automatic 4x4                 3
Automatic Tipshift            2
Auto Steptronic               2
Electronic Automatic          2
Automatic Steptronic          1
Auto Tip

In [678]:
# Binning transmission values based on "Auto|Automatic" and "Manual"
df_clean.loc[df_clean['Transmission'].str.contains('auto|automatic', case=False), 'Transmission'] = 'Automatic'
df_clean.loc[df_clean['Transmission'].str.contains('manual', case=False), 'Transmission'] = 'Manual'
df_clean['Transmission'].value_counts()

Automatic              1467
Continuous Variable     344
Manual                  312
Direct Shift             26
-                         6
CVT Multitronic           3
Name: Transmission, dtype: int64

In [679]:
# Binning transmission values for items < 300
transmission = df_clean['Transmission'].value_counts()
transmission_replace = list(transmission[transmission < 300].index)

for trans in transmission_replace:
    df_clean['Transmission'] = df_clean['Transmission'].replace(trans,"Other")

df_clean['Transmission'].value_counts()

Automatic              1467
Continuous Variable     344
Manual                  312
Other                    35
Name: Transmission, dtype: int64

In [680]:
# Validate for null values found in the previous result
print(df_clean['Transmission'].isnull().value_counts())

False    2158
Name: Transmission, dtype: int64


In [681]:
# Extract the body type only and rmeove seats and doors, and adjust capital letters
df_clean['Body type'] = df_clean['Body type'].str.split(',').str[0].str.strip()
df_clean['Body type'] = df_clean['Body type'].str.lower().str.capitalize()
body_type = df_clean['Body type'].value_counts()
body_type

Suv             944
Ute tray        424
Hatch           386
Sedan           230
Wagon           103
Commercial       29
Coupe            28
Convertible       8
People mover      5
Other             1
Name: Body type, dtype: int64

In [682]:
# Binning the body types < 20
body_type_replace = list(body_type[body_type < 20].index)

for body in body_type_replace:
    df_clean['Body type'] = df_clean['Body type'].replace(body,"Other")

df_clean['Body type'].value_counts()

Suv           944
Ute tray      424
Hatch         386
Sedan         230
Wagon         103
Commercial     29
Coupe          28
Other          14
Name: Body type, dtype: int64

In [683]:
# Execute value counts for Fuel type for binning
fuel_type = df_clean['Fuel type'].value_counts()
fuel_type

Unleaded Petrol              927
Diesel                       788
Premium Unleaded Petrol      372
Unleaded Petrol/Electric      48
Premium Unleaded/Electric     20
Diesel/Electric                1
Liquid Petroleum Gas           1
Electric/Pulp                  1
Name: Fuel type, dtype: int64

In [684]:
# Binning the fuel type < 300
fuel_type_replace = list(fuel_type[fuel_type < 130].index)

for fuel in fuel_type_replace:
    df_clean['Fuel type'] = df_clean['Fuel type'].replace(fuel,"Other")

df_clean['Fuel type'].value_counts()

Unleaded Petrol            927
Diesel                     788
Premium Unleaded Petrol    372
Other                       71
Name: Fuel type, dtype: int64

In [685]:
# Validate for null values found in the previous result
print(df_clean['Fuel type'].isnull().value_counts())

False    2158
Name: Fuel type, dtype: int64


In [686]:
# Adjust drive type to change Four Wheel Drive to 4x4
df_clean['Drive type'] = df_clean['Drive type'].replace('4x4', 'Four Wheel Drive')
df_clean['Drive type'] = df_clean['Drive type'].replace('4x2', 'Rear Wheel Drive')
df_clean['Drive type'].value_counts()

Front Wheel Drive    893
Four Wheel Drive     617
All Wheel Drive      397
Rear Wheel Drive     251
Name: Drive type, dtype: int64

In [687]:
# Validate for null values found in the previous result
print(df_clean['Drive type'].isnull().value_counts())

False    2158
Name: Drive type, dtype: int64


In [688]:
# Extract the fuel consumption liters part
df_clean['Fuel consumption'] = df_clean['Fuel consumption'].str.extract(r'^(\d+(\.\d+)?)\s').astype('float64')
df_clean = df_clean.rename(columns={'Fuel consumption': 'Fuel consumption per 100km'})
df_clean.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption per 100km,Colour ext / int,...,CC,Number of cylinders,Fuel tank capacity,Valve gear type,Maximum torque,Maximum power (kW),Kerb weight,Make,Model,Variant
1,2019,2019 Hyundai I30 Active PD2 MY19,74070.0,20998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
2,2019,2019 Hyundai I30 Active PD2 MY19,66975.0,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
3,2019,2019 Hyundai I30 Active PD2 MY19,68845.0,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver / -,...,1999.0,4.0,50.0,16.0,203 @ 4700 RPM,120 @ 6200 RPM,1276 kg,Hyundai,I30,Active
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,188978.0,24750,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White / Black,...,2440.0,4.0,75.0,16.0,430 @ 2500 RPM,133 @ 3500 RPM,1930 kg,Mitsubishi,Triton,GLX (4X4)
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,141855.0,26900,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White / Black,...,2440.0,4.0,75.0,16.0,430 @ 2500 RPM,133 @ 3500 RPM,1930 kg,Mitsubishi,Triton,GLX (4X4)


In [689]:
# Validate for null values found in the previous result
print(df_clean['Fuel consumption per 100km'].isnull().value_counts())

False    2158
Name: Fuel consumption per 100km, dtype: int64


In [690]:
# Adjust colour coloumn to only ext colour
df_clean['Colour ext / int'] = df_clean['Colour ext / int'].str.split().str[0]
df_clean = df_clean.rename(columns={'Colour ext / int': 'Colour ext'})
colour_ext = df_clean['Colour ext'].value_counts()
colour_ext

White       923
Silver      308
Grey        293
Black       195
Blue        190
Red         132
Green        24
-            23
Brown        21
Orange       20
Gold         12
Yellow       10
Beige         5
Burgundy      2
Name: Colour ext, dtype: int64

In [691]:
# Binning the colours < 130
colour_ext_replace = list(colour_ext[colour_ext < 130].index)

for colour in colour_ext_replace:
    df_clean['Colour ext'] = df_clean['Colour ext'].replace(colour,"Other")

df_clean['Colour ext'].value_counts()

White     923
Silver    308
Grey      293
Black     195
Blue      190
Red       132
Other     117
Name: Colour ext, dtype: int64

In [692]:
# Validate for null values found in the previous result
print(df_clean['Colour ext'].isnull().value_counts())

False    2158
Name: Colour ext, dtype: int64


In [693]:
# Check datatypes
df_clean.dtypes

Year model                      int64
Car Spec                       object
Kilometres                    float64
Price                           int64
Transmission                   object
Body type                      object
Drive type                     object
Fuel type                      object
Fuel consumption per 100km    float64
Colour ext                     object
VIN                            object
Dealer                         object
Address                        object
Seating capacity              float64
Doors                         float64
CC                            float64
Number of cylinders           float64
Fuel tank capacity            float64
Valve gear type               float64
Maximum torque                 object
Maximum power (kW)             object
Kerb weight                    object
Make                           object
Model                          object
Variant                        object
dtype: object

In [694]:
# Change datatypes of some numerical columns from float to integers
columns_to_convert = ['Kilometres', 'Seating capacity', 'Doors','CC', 'Number of cylinders', 'Fuel tank capacity', 'Valve gear type']
df_clean[columns_to_convert] = df_clean[columns_to_convert].astype('int64')
df_clean.dtypes

Year model                      int64
Car Spec                       object
Kilometres                      int64
Price                           int64
Transmission                   object
Body type                      object
Drive type                     object
Fuel type                      object
Fuel consumption per 100km    float64
Colour ext                     object
VIN                            object
Dealer                         object
Address                        object
Seating capacity                int64
Doors                           int64
CC                              int64
Number of cylinders             int64
Fuel tank capacity              int64
Valve gear type                 int64
Maximum torque                 object
Maximum power (kW)             object
Kerb weight                    object
Make                           object
Model                          object
Variant                        object
dtype: object

In [695]:
# Validate for null values found in the previous result
print(df_clean.isnull().sum())

Year model                    0
Car Spec                      0
Kilometres                    0
Price                         0
Transmission                  0
Body type                     0
Drive type                    0
Fuel type                     0
Fuel consumption per 100km    0
Colour ext                    0
VIN                           0
Dealer                        0
Address                       0
Seating capacity              0
Doors                         0
CC                            0
Number of cylinders           0
Fuel tank capacity            0
Valve gear type               0
Maximum torque                0
Maximum power (kW)            0
Kerb weight                   0
Make                          0
Model                         0
Variant                       0
dtype: int64


In [696]:
# From the Max Torque and Power, extract only the Peak torque (first value of Max Torque) and the Horse Power first value of Max Power converted from KW to HP)
df_clean['Maximum torque'] = df_clean['Maximum torque'].str.split().str[0]
df_clean = df_clean.rename(columns={'Maximum torque': 'Peak torque (Nm)'})
df_clean['Peak torque (Nm)'] = df_clean['Peak torque (Nm)'].astype('int64')

df_clean['Maximum power (kW)'] = df_clean['Maximum power (kW)'].str.split().str[0]
df_clean['Maximum power (kW)'] = pd.to_numeric(df_clean['Maximum power (kW)'], errors='coerce')

conversion_factor_kW_HP =  1.341
df_clean['Maximum power (kW)'] = df_clean['Maximum power (kW)'] * conversion_factor_kW_HP
df_clean = df_clean.rename(columns={'Maximum power (kW)': 'Horsepower'})
df_clean['Horsepower'] = df_clean['Horsepower'].apply(math.ceil)

df_clean.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption per 100km,Colour ext,...,CC,Number of cylinders,Fuel tank capacity,Valve gear type,Peak torque (Nm),Horsepower,Kerb weight,Make,Model,Variant
1,2019,2019 Hyundai I30 Active PD2 MY19,74070,20998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,1999,4,50,16,203,161,1276 kg,Hyundai,I30,Active
2,2019,2019 Hyundai I30 Active PD2 MY19,66975,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,1999,4,50,16,203,161,1276 kg,Hyundai,I30,Active
3,2019,2019 Hyundai I30 Active PD2 MY19,68845,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,1999,4,50,16,203,161,1276 kg,Hyundai,I30,Active
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,188978,24750,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,2440,4,75,16,430,179,1930 kg,Mitsubishi,Triton,GLX (4X4)
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,141855,26900,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,2440,4,75,16,430,179,1930 kg,Mitsubishi,Triton,GLX (4X4)


In [697]:
# Validate for null values found in the previous result
print(df_clean['Peak torque (Nm)'].isnull().value_counts())
print(df_clean['Horsepower'].isnull().value_counts())

False    2158
Name: Peak torque (Nm), dtype: int64
False    2158
Name: Horsepower, dtype: int64


In [698]:
# Convert Kerb weight from string to integer
df_clean['Kerb weight'] = df_clean['Kerb weight'].str.split().str[0]
df_clean['Kerb weight'] = pd.to_numeric(df_clean['Kerb weight'], errors='coerce')
df_clean['Kerb weight'] = df_clean['Kerb weight'].astype('Int64')
df_clean.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption per 100km,Colour ext,...,CC,Number of cylinders,Fuel tank capacity,Valve gear type,Peak torque (Nm),Horsepower,Kerb weight,Make,Model,Variant
1,2019,2019 Hyundai I30 Active PD2 MY19,74070,20998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,1999,4,50,16,203,161,1276,Hyundai,I30,Active
2,2019,2019 Hyundai I30 Active PD2 MY19,66975,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,1999,4,50,16,203,161,1276,Hyundai,I30,Active
3,2019,2019 Hyundai I30 Active PD2 MY19,68845,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,1999,4,50,16,203,161,1276,Hyundai,I30,Active
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,188978,24750,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,2440,4,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4)
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,141855,26900,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,2440,4,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4)


In [699]:
# Validate for null values found in the previous result
print(df_clean['Kerb weight'].isnull().value_counts())

False    2156
True        2
Name: Kerb weight, dtype: int64


In [700]:
# Drop the null values
df_clean.dropna(inplace=True)

In [701]:
# Last drop of rows where values are "0" and any value wih "-"
df_clean = df_clean.loc[(df_clean != 0).all(axis=1)]
df_clean = df_clean.replace("-", float("nan")).dropna()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 1 to 5669
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year model                  2126 non-null   int64  
 1   Car Spec                    2126 non-null   object 
 2   Kilometres                  2126 non-null   int64  
 3   Price                       2126 non-null   int64  
 4   Transmission                2126 non-null   object 
 5   Body type                   2126 non-null   object 
 6   Drive type                  2126 non-null   object 
 7   Fuel type                   2126 non-null   object 
 8   Fuel consumption per 100km  2126 non-null   float64
 9   Colour ext                  2126 non-null   object 
 10  VIN                         2126 non-null   object 
 11  Dealer                      2126 non-null   object 
 12  Address                     2126 non-null   object 
 13  Seating capacity            2126 

In [702]:
# Double checl for null values found in the previous data cleaning exercise
print(df_clean.isnull().sum())

Year model                    0
Car Spec                      0
Kilometres                    0
Price                         0
Transmission                  0
Body type                     0
Drive type                    0
Fuel type                     0
Fuel consumption per 100km    0
Colour ext                    0
VIN                           0
Dealer                        0
Address                       0
Seating capacity              0
Doors                         0
CC                            0
Number of cylinders           0
Fuel tank capacity            0
Valve gear type               0
Peak torque (Nm)              0
Horsepower                    0
Kerb weight                   0
Make                          0
Model                         0
Variant                       0
dtype: int64


In [703]:
# Last check of all datatypes
df_clean.dtypes

Year model                      int64
Car Spec                       object
Kilometres                      int64
Price                           int64
Transmission                   object
Body type                      object
Drive type                     object
Fuel type                      object
Fuel consumption per 100km    float64
Colour ext                     object
VIN                            object
Dealer                         object
Address                        object
Seating capacity                int64
Doors                           int64
CC                              int64
Number of cylinders             int64
Fuel tank capacity              int64
Valve gear type                 int64
Peak torque (Nm)                int64
Horsepower                      int64
Kerb weight                     Int64
Make                           object
Model                          object
Variant                        object
dtype: object

In [704]:
# Assign the new df_clean to a new variable and download a file to check all data is correct
df_car_listing = df_clean
df_car_listing.to_csv('data_cleaning_output/used_cars_dataset_test.csv', index=False)
print("Dataset test save: Completed")

Dataset test save: Completed


In [705]:
## Add columns to the df where by using the Geoapify API will fetch latitude and longitude
df_car_listing["lat"] = ""
df_car_listing["lon"] = ""
df_car_listing.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption per 100km,Colour ext,...,Fuel tank capacity,Valve gear type,Peak torque (Nm),Horsepower,Kerb weight,Make,Model,Variant,lat,lon
1,2019,2019 Hyundai I30 Active PD2 MY19,74070,20998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,,
2,2019,2019 Hyundai I30 Active PD2 MY19,66975,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,,
3,2019,2019 Hyundai I30 Active PD2 MY19,68845,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,,
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,188978,24750,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4),,
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,141855,26900,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4),,


In [706]:
# Define the API parameters to get the Geoapify link
base_url = "https://api.geoapify.com/v1/geocode/search?"
params = {
    "apiKey": geoapify_key,
    "format": "json"
}

In [707]:
# Print a message to follow up the lat and lon search
print("Starting lat and lon search:")
print("----------------------------")

# Loop through the customers_df DataFrame and search coordinates for each customer
for i, row in df_car_listing.iterrows():
    params["text"] = f'{row["Address"]}'

    # Make the API request
    response = requests.get(base_url, params=params)

    # Convert reponse to JSON
    response = response.json()

    # Extract latitude and longitude
    lat = response["results"][0]["lat"]
    lon = response["results"][0]["lon"]

    # Add lat and lon to the customers_df DataFrame
    df_car_listing.loc[i, "lat"] = lat
    df_car_listing.loc[i, "lon"] = lon

    # Log the search results
    print(f"Coordinates for {row['Address']} fetched. Lat: {lat}, Lon: {lon}.")

# Display sample data to confirm that the coordinates appear
df_car_listing.head()


Starting lat and lon search:
----------------------------
Coordinates for 1324 Albany Highway, Cannington, WA fetched. Lat: -32.0166894, Lon: 115.9322155.
Coordinates for 1324 Albany Highway, Cannington, WA fetched. Lat: -32.0166894, Lon: 115.9322155.
Coordinates for 1324 Albany Highway, Cannington, WA fetched. Lat: -32.0166894, Lon: 115.9322155.
Coordinates for 76 Division Street, Welshpool, WA fetched. Lat: -31.987188, Lon: 115.935923.
Coordinates for 76 Division Street, Welshpool, WA fetched. Lat: -31.987188, Lon: 115.935923.
Coordinates for 170 Welshpool Road, Welshpool, WA fetched. Lat: -31.995227, Lon: 115.938215.
Coordinates for 76 Division Street, Welshpool, WA fetched. Lat: -31.987188, Lon: 115.935923.
Coordinates for 464 Canning Highway, Como, WA fetched. Lat: -32.00604, Lon: 115.8597.
Coordinates for 76 Division Street, Welshpool, WA fetched. Lat: -31.987188, Lon: 115.935923.
Coordinates for 1093 Albany Highway, Bentley, WA fetched. Lat: -31.999547, Lon: 115.916029.
Coordina

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption per 100km,Colour ext,...,Fuel tank capacity,Valve gear type,Peak torque (Nm),Horsepower,Kerb weight,Make,Model,Variant,lat,lon
1,2019,2019 Hyundai I30 Active PD2 MY19,74070,20998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,-32.0167,115.932
2,2019,2019 Hyundai I30 Active PD2 MY19,66975,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,-32.0167,115.932
3,2019,2019 Hyundai I30 Active PD2 MY19,68845,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,-32.0167,115.932
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,188978,24750,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4),-31.9872,115.936
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,141855,26900,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4),-31.9872,115.936


In [708]:
# Check for null values found in the previous data cleaning exercise
print(df_car_listing.isnull().sum())

Year model                    0
Car Spec                      0
Kilometres                    0
Price                         0
Transmission                  0
Body type                     0
Drive type                    0
Fuel type                     0
Fuel consumption per 100km    0
Colour ext                    0
VIN                           0
Dealer                        0
Address                       0
Seating capacity              0
Doors                         0
CC                            0
Number of cylinders           0
Fuel tank capacity            0
Valve gear type               0
Peak torque (Nm)              0
Horsepower                    0
Kerb weight                   0
Make                          0
Model                         0
Variant                       0
lat                           0
lon                           0
dtype: int64


In [709]:
# Change lat and lon to float
df_car_listing_complete = df_car_listing.astype({'lat':'float', 'lon': 'float'})
df_car_listing_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 1 to 5669
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year model                  2126 non-null   int64  
 1   Car Spec                    2126 non-null   object 
 2   Kilometres                  2126 non-null   int64  
 3   Price                       2126 non-null   int64  
 4   Transmission                2126 non-null   object 
 5   Body type                   2126 non-null   object 
 6   Drive type                  2126 non-null   object 
 7   Fuel type                   2126 non-null   object 
 8   Fuel consumption per 100km  2126 non-null   float64
 9   Colour ext                  2126 non-null   object 
 10  VIN                         2126 non-null   object 
 11  Dealer                      2126 non-null   object 
 12  Address                     2126 non-null   object 
 13  Seating capacity            2126 

In [713]:
# Check datatypes
df_car_listing_complete.dtypes

Year model                      int64
Car Spec                       object
Kilometres                      int64
Price                           int64
Transmission                   object
Body type                      object
Drive type                     object
Fuel type                      object
Fuel consumption per 100km    float64
Colour ext                     object
VIN                            object
Dealer                         object
Address                        object
Seating capacity                int64
Doors                           int64
CC                              int64
Number of cylinders             int64
Fuel tank capacity              int64
Valve gear type                 int64
Peak torque (Nm)                int64
Horsepower                      int64
Kerb weight                     Int64
Make                           object
Model                          object
Variant                        object
lat                           float64
lon         

In [714]:
# Save the new cleaned dataset
df_car_listing_complete.to_csv('data_cleaning_output/used_cars_dataset_loc.csv', index=False)

In [724]:
# Filter the dataframe based on the latitude and longitude conditions
lat_condition = -30
filtered_df = df_car_listing_complete[(df_car_listing_complete['lat'] > lat_condition) & (df_car_listing_complete['lon'] < 115)]

# Retrieve the list of addresses from the filtered dataframe
address_list = filtered_df[['Dealer', 'Address']].drop_duplicates().values.tolist()

# Print the list of dealer name and address
for dealer_name, address in address_list:
    print(f"Dealer: {dealer_name}, Address: {address}")


Dealer: Auto Planet WA, Address: 468 Albany Highway, Victoria Park, WA
Dealer: John Hughes Passenger, Address: 196 Albany Highway, Victoria Park, WA
Dealer: John Hughes Group - Auto Warehouse, Address: Albany Highway, Victoria Park, WA
Dealer: Youngs WA, Address: 529 Albany Highway, Victoria Park, WA
Dealer: John Hughes Commercials, Address: 234 Albany Highway, Victoria Park, WA
Dealer: John Hughes Group Volkswagen, Address: 61 Shepperton Road, Victoria Park, WA
Dealer: Cut Price Car Co, Address: 688 Albany Highway, Victoria Park, WA
Dealer: Perth City Subaru, Address: 165 Albany Highway, Victoria Park, WA
Dealer: Perth City Prestige, Address: 495 Albany Highway, Victoria Park, WA


In [727]:
# Manually update the latitude and longitude values for each address
updated_lat_lon = {
    '468 Albany Highway, Victoria Park, WA': (-31.976168, 115.898473),
    '196 Albany Highway, Victoria Park, WA': (-31.9705, 115.89301),
    'Albany Highway, Victoria Park, WA': (-31.970971650000003, 115.89225179909646),
    '529 Albany Highway, Victoria Park, WA': (-31.978105, 115.89843),
    '234 Albany Highway, Victoria Park, WA': (-31.9714252, 115.8939466),
    '61 Shepperton Road, Victoria Park, WA': (-31.970769699999998, 115.89495889583115),
    '688 Albany Highway, Victoria Park, WA': (-31.98124, 115.9013),
    '165 Albany Highway, Victoria Park, WA': (-31.9704221, 115.89155790526316),
    '495 Albany Highway, Victoria Park, WA': (-31.97739, 115.89798),
}

In [731]:
# Create a copy of the original DataFrame
df_updated = df_car_listing_complete.copy()

# Update the latitude and longitude values in the copied DataFrame
for dealer_name, address in address_list:
    if address in updated_lat_lon:
        new_lat, new_lon = updated_lat_lon[address]
        df_updated.loc[(df_updated['Address'] == address), 'lat'] = new_lat
        df_updated.loc[(df_updated['Address'] == address), 'lon'] = new_lon

# Validate the changes in the new df_updated
df_updated.head()

,Year model,Car Spec,Kilometres,Price,Transmission,Body type,Drive type,Fuel type,Fuel consumption per 100km,Colour ext,...,Fuel tank capacity,Valve gear type,Peak torque (Nm),Horsepower,Kerb weight,Make,Model,Variant,lat,lon
1,2019,2019 Hyundai I30 Active PD2 MY19,74070,20998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,-32.016689,115.932215
2,2019,2019 Hyundai I30 Active PD2 MY19,66975,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,-32.016689,115.932215
3,2019,2019 Hyundai I30 Active PD2 MY19,68845,21998,Automatic,Hatch,Front Wheel Drive,Unleaded Petrol,7.4,Silver,...,50,16,203,161,1276,Hyundai,I30,Active,-32.016689,115.932215
4,2017,2017 Mitsubishi Triton GLX (4X4) MQ MY18,188978,24750,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4),-31.987188,115.935923
5,2016,2016 Mitsubishi Triton GLX (4X4) MQ MY16,141855,26900,Automatic,Ute tray,Four Wheel Drive,Diesel,7.6,White,...,75,16,430,179,1930,Mitsubishi,Triton,GLX (4X4),-31.987188,115.935923


In [736]:
# Get the list of addresses from the dictionary keys
addresses = list(updated_lat_lon.keys())

# Filter the original DataFrame based on the addresses
filtered_df = df_updated[df_updated['Address'].isin(addresses)]

# Display the filtered addresses, latitude, and longitude
filtered_addresses = filtered_df[['Address', 'lat', 'lon']].drop_duplicates()
filtered_addresses

,Address,lat,lon
268,"468 Albany Highway, Victoria Park, WA",-31.976168,115.898473
531,"196 Albany Highway, Victoria Park, WA",-31.970500,115.893010
534,"Albany Highway, Victoria Park, WA",-31.970972,115.892252
1086,"529 Albany Highway, Victoria Park, WA",-31.978105,115.898430
1653,"234 Albany Highway, Victoria Park, WA",-31.971425,115.893947
2051,"61 Shepperton Road, Victoria Park, WA",-31.970770,115.894959
2687,"688 Albany Highway, Victoria Park, WA",-31.981240,115.901300
2788,"165 Albany Highway, Victoria Park, WA",-31.970422,115.891558
4838,"495 Albany Highway, Victoria Park, WA",-31.977390,115.897980


In [738]:
# Validate datatypes
df_updated.dtypes

Year model                      int64
Car Spec                       object
Kilometres                      int64
Price                           int64
Transmission                   object
Body type                      object
Drive type                     object
Fuel type                      object
Fuel consumption per 100km    float64
Colour ext                     object
VIN                            object
Dealer                         object
Address                        object
Seating capacity                int64
Doors                           int64
CC                              int64
Number of cylinders             int64
Fuel tank capacity              int64
Valve gear type                 int64
Peak torque (Nm)                int64
Horsepower                      int64
Kerb weight                     Int64
Make                           object
Model                          object
Variant                        object
lat                           float64
lon         

In [740]:
# Rename lat and lon with a capital letter
df_updated = df_updated.rename(columns={'lat': 'Lat', 'lon': 'Lon'})

# Validate data info
df_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 1 to 5669
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year model                  2126 non-null   int64  
 1   Car Spec                    2126 non-null   object 
 2   Kilometres                  2126 non-null   int64  
 3   Price                       2126 non-null   int64  
 4   Transmission                2126 non-null   object 
 5   Body type                   2126 non-null   object 
 6   Drive type                  2126 non-null   object 
 7   Fuel type                   2126 non-null   object 
 8   Fuel consumption per 100km  2126 non-null   float64
 9   Colour ext                  2126 non-null   object 
 10  VIN                         2126 non-null   object 
 11  Dealer                      2126 non-null   object 
 12  Address                     2126 non-null   object 
 13  Seating capacity            2126 

In [741]:
# Save the final dataset
df_updated.to_csv('data_cleaning_output/used_cars_dataset_complete.csv', index=False)